# todo: import 3 datasets, generate 3 adata

import dataset by relative path style

In [ ]:
import pandas as pd
import os
import anndata as ad

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.manifold import TSNE
from umap import UMAP

RELATIVE_DATA_PATH = os.path.join(os.path.pardir, 'data')

DATA_SET_NAME = 'german_socio_eco'

# Load data
data_path = os.path.join(RELATIVE_DATA_PATH, DATA_SET_NAME, f'{DATA_SET_NAME}.csv')
df_data = pd.read_csv(data_path)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_data)
adata = ad.AnnData(X = data_scaled)
adata.layers['raw'] = df_data.values
adata.layers['scaled'] = data_scaled
adata.var_names = df_data.columns.astype(str)
# todo: tweak the below to be compatible with all data sets
adata.var['var_type'] = 'numeric'
# compute tsne embeddings of data_scaled
tsne = TSNE(n_components=2)
adata.obsm['tsne'] = tsne.fit_transform(data_scaled)
pca = PCA(n_components=2)
adata.obsm['pca'] = pca.fit_transform(data_scaled)
# todo: fix umap to work successfully
umap = UMAP(n_components=2)
adata.obsm['umap'] = umap.fit_transform(data_scaled)
# todo: import Edith's code to compute example tsne embeddings
# todo: write obsm - embeddings
adata.write(os.path.join(RELATIVE_DATA_PATH, DATA_SET_NAME, f'{DATA_SET_NAME}.h5ad'))


c:\Users\Administrator\anaconda3\envs\ExClus\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: <code object <module> at 0x000002277ED156F0, file "c:\Users\Administrator\anaconda3\envs\ExClus\Lib\site-packages\pynndescent\sparse_nndescent.py", line 1> != <code object make_heap at 0x000002277E959790, file "c:\Users\Administrator\anaconda3\envs\ExClus\Lib\site-packages\pynndescent\utils.py", line 171>

TODO: generate adata

In [ ]:
from sklearn.discriminant_analysis import StandardScaler


def generate_adata(dataset_name):
    '''
    Data preprocessing: given dataset name, generating adata file contain versions of dataset, feature types and embeddings
    :param DATA_SET_NAME:
    :return:
    '''
    data_path = os.path.join(RELATIVE_DATA_PATH, dataset_name, f'{dataset_name}.csv')
    df_data = pd.read_csv(data_path)
    # scaler fit transform df_data
    scaler = StandardScaler()
    hd_data = scaler.fit_transform(df_data)
    adata = ad.AnnData(X=hd_data)